<a href="https://colab.research.google.com/github/salmanromeo/MAE_3403_Computer_Methods_in_Analysis_and_Design/blob/main/lecture_9_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Optimization (scipy.optimize)**

#####The **scipy.optimize** package provides several commonly used optimization algorithms.

**Unconstrained minimization of multivariate scalar functions (minimize)**

#####The **minimize** function provides a common interface to unconstrained and constrained minimization algorithms for multivariate scalar functions in scipy.optimize. To demonstrate the minimization function, consider the problem of minimizing the **Rosenbrock function** of $N$ variables:
\begin{align}
  f(x) = \sum_{i=1}^{N-1} 100(x_{i+1}-x_i^2)^2 + (1-x_i)^2
    \end{align}
#####The minimum value of this function is $0$ which is achieved when $x_i=1$.

**1a. Nelder-Mead Simplex algorithm (method='Nelder-Mead')**

In [1]:
import numpy as np
from scipy.optimize import minimize

# define the Rosenbrock function
def rosen(x):
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

# define an initial guess for the optimization algorithm
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

res = minimize(rosen, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 339
         Function evaluations: 571
[1. 1. 1. 1. 1.]


#####The simplex algorithm is probably the simplest way to minimize a fairly well-behaved function. It requires only function evaluations and is a good choice for simple minimization problems. However, because it does not use any gradient evaluations, it may take longer to find the minimum.

**1b. Nelder-Mead Simplex algorithm (method='powell')**

#####Another optimization algorithm that needs only function calls to find the minimum is Powell’s method available by setting **method='powell'** in minimize.

In [3]:
import numpy as np
from scipy.optimize import minimize

# define the Rosenbrock function
def rosen(x):
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

# define an initial guess for the optimization algorithm
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

res = minimize(rosen, x0, method='powell',
               options={'disp': True})

print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 18
         Function evaluations: 988
[1. 1. 1. 1. 1.]


**1c. Rosenbrock function with an additional scaling factor $a$ and an offset $b$**

\begin{align}
  f(x,a,b) = \sum_{i=1}^{N-1} a(x_{i+1}-x_i^2)^2 + (1-x_i)^2 + b
    \end{align}
#####Example parameters $a=0.5$ and $b=1$.

In [4]:
import numpy as np
from scipy.optimize import minimize

# define the Rosenbrock function with additional arguments
def rosen_with_args(x, a, b):
    return sum(a*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0) + b

# define an initial guess for the optimization algorithm
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

res = minimize(rosen_with_args, x0, method='nelder-mead',
               args=(0.5, 1.), options={'xatol': 1e-8, 'disp': True})

print(res.x)

Optimization terminated successfully.
         Current function value: 1.000000
         Iterations: 319
         Function evaluations: 525
[1.         1.         1.         1.         0.99999999]


**2a. Broyden-Fletcher-Goldfarb-Shanno algorithm (method='BFGS')**

#####In order to converge more quickly to the solution, this routine uses the gradient of the objective function.
#####To demonstrate this algorithm, the Rosenbrock function is again used. The gradient of the Rosenbrock function is the vector:
\begin{align}
  \frac{∂f}{∂x_j} = \sum_{i=1}^{N} 200(x_{i}-x_{i-1}^2)(δ_{i,j}-2x_{i-1}δ_{i-1,j}) - 2(1-x_{i-1})δ_{i-1,j} \\[1em]
  = 200(x_{j}-x_{j-1}^2) -400x_j(x_{j+1}-x_j^2)- 2(1-x_{j})
    \end{align}
#####This expression is valid for the interior derivatives. Special cases are
\begin{align}
  \frac{∂f}{∂x_0} = -400x_0(x_1-x_0^2)- 2(1-x_{0}) \\[1em]
    \frac{∂f}{∂x_{N-1}} = 200x_0(x_{N-1}-x_{N-2}^2)
    \end{align}

In [5]:
import numpy as np
from scipy.optimize import minimize

# define the Rosenbrock function
def rosen(x):
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

# define derivatives
def rosen_der(x):
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)
    return der

# define an initial guess for the optimization algorithm
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

res = minimize(rosen, x0, tol = 1e-8, method='BFGS', jac=rosen_der,
               options={'disp': True})

print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 28
         Function evaluations: 33
         Gradient evaluations: 33
[1. 1. 1. 1. 1.]


**2b. Broyden-Fletcher-Goldfarb-Shanno algorithm (method='BFGS', jac=True)**

#####Another way to supply gradient information is to write a single function which returns both the objective and the gradient: this is indicated by setting **jac=True**.
#####In this case, the Python function to be optimized must return a tuple whose first value is the objective and whose second value represents the gradient.

In [6]:
import numpy as np
from scipy.optimize import minimize

# define the Rosenbrock function
def rosen(x):
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

# define derivatives
def rosen_and_der(x):
    objective = sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)
    return objective, der

# define an initial guess for the optimization algorithm
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

res = minimize(rosen_and_der, x0, tol = 1e-8, method='BFGS', jac=True,
               options={'disp': True})

print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 28
         Function evaluations: 33
         Gradient evaluations: 33
[1. 1. 1. 1. 1.]


#####Supplying objective and gradient in a single function can help to avoid redundant computations and therefore speed up the optimization significantly.

**3. Newton-Conjugate-Gradient algorithm (method='Newton-CG')**

#####Newton-Conjugate Gradient algorithm is a modified Newton’s method and uses a conjugate gradient algorithm to (approximately) invert the local Hessian. Newton’s method is based on fitting the function locally to a quadratic form:
\begin{align}
  f(x) ≈ f(x_0)+∇f(x_0)(x-x_0)+\frac{1}{2}(x-x_0)^T𝐇(x_0)(x-x_0)
    \end{align}
#####where $𝐇(x_0)$ is a matrix of second-derivatives (the Hessian). If the Hessian is positive definite then the local minimum of this function can be found by setting the gradient of the quadratic form to zero, resulting in
\begin{align}
  x_{opt} = x_0-𝐇^{-1}∇f
    \end{align}
#####The inverse of the Hessian is evaluated using the conjugate-gradient method. An example of employing this method to minimizing the Rosenbrock function is given below.
#####The Hessian of the Rosenbrock function is
\begin{align}
  𝐇_{i,j} = \frac{∂^2f}{∂x_i∂x_j} = 200(δ_{i,j}-2x_{i-1}δ_{i-1,j})-400x_i(∂_{i+1,j}-2x_iδ_{i,j})-400δ_{i,j}(x_{i+1}-x_{i}^2)+2δ_{i,j} \\[1em]
  =(202+1200x_{i}^2-400x_{i+1})δ_{i,j}-400x_iδ_{i+1,j}-400x_{i-1}δ_{i-1,j}
    \end{align}
#####if $i,j ∈ [1,N-2]$ with $i,j ∈ [0,N-1]$ defining the $NxN$ matrix. Other non-zero entries of the matrix are
\begin{align}
  \frac{∂^2f}{∂x_0^2} = 1200x_0-400x_1+2; \\[1em]
  \frac{∂^2f}{∂x_0∂x_1} = \frac{∂^2f}{∂x_1∂x_0} =-400x_0; \\[1em]
  \frac{∂^2f}{∂x_{N-1}∂x_{N-2}} = \frac{∂^2f}{∂x_{N-2}∂x_{N-1}} =-400x_{N-2};  \\[1em]
  \frac{∂^2f}{∂x_{N-1}^2} = 200.
    \end{align}


#####For example, the Hessian when $N=5$ is
\begin{align}
𝐇 = \begin{bmatrix}
1200x_0-400x_1+2 & -400x_0 & 0 & 0 & 0\\
-400x_0 & 202+1200x_1^2-400x_2 & -400x_1 & 0 & 0\\
0 & -400x_1 & 202+1200x_2^2-400x_3 & -400x_2 & 0\\
0 & 0 & -400x_2 & 202+1200x_3^2-400x_4 & -400x_3 \\
0 & 0 & 0 & -400x_3 & 200
\end{bmatrix}
\end{align}

#####The code which computes this Hessian along with the code to minimize the function using Newton-CG method is shown in the following example:

In [7]:
import numpy as np
from scipy.optimize import minimize

# define the Rosenbrock function
def rosen(x):
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

# define derivatives
def rosen_der(x):
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)
    return der

# define Hessian
def rosen_hess(x):
    x = np.asarray(x)
    H = np.diag(-400*x[:-1],1) - np.diag(400*x[:-1],-1)
    diagonal = np.zeros_like(x)
    diagonal[0] = 1200*x[0]**2-400*x[1]+2
    diagonal[-1] = 200
    diagonal[1:-1] = 202 + 1200*x[1:-1]**2 - 400*x[2:]
    H = H + np.diag(diagonal)
    return H

# define an initial guess for the optimization algorithm
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

res = minimize(rosen, x0, method='Newton-CG',
               jac=rosen_der, hess=rosen_hess,
               options={'xtol': 1e-8, 'disp': True})

print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 24
         Function evaluations: 33
         Gradient evaluations: 33
         Hessian evaluations: 24
[1.         1.         1.         0.99999999 0.99999999]


**4. Trust-Region Newton-Conjugate-Gradient Algorithm (method='trust-ncg')**

#####The Newton-CG method is a line search method: it finds a direction of search minimizing a quadratic approximation of the function and then uses a line search algorithm to find the (nearly) optimal step size in that direction. An alternative approach is to, first, fix the step size limit $𝚫$ and then find the optimal step $p$ inside the given trust-radius by solving the following quadratic subproblem:
\begin{align}
  \text{min}_{p} f(x_k) + ∇f(x_k).p+\frac{1}{2}p^T𝐇(x_k)p \\[1em]
  \text{subject to:} \mid\mid p\mid\mid \le 𝚫
    \end{align}
#####The solution is then updated \begin{align}
  x_{k+1} = x_k+p
    \end{align}
 and the trust-radius $𝚫$ is adjusted according to the degree of agreement of the quadratic model with the real function. This family of methods is known as trust-region methods. The trust-ncg algorithm is a trust-region method that uses a conjugate gradient algorithm to solve the trust-region subproblem.

In [8]:
import numpy as np
from scipy.optimize import minimize

# define the Rosenbrock function
def rosen(x):
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

# define derivatives
def rosen_der(x):
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)
    return der

# define Hessian
def rosen_hess(x):
    x = np.asarray(x)
    H = np.diag(-400*x[:-1],1) - np.diag(400*x[:-1],-1)
    diagonal = np.zeros_like(x)
    diagonal[0] = 1200*x[0]**2-400*x[1]+2
    diagonal[-1] = 200
    diagonal[1:-1] = 202 + 1200*x[1:-1]**2 - 400*x[2:]
    H = H + np.diag(diagonal)
    return H

# define an initial guess for the optimization algorithm
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

res = minimize(rosen, x0, method='trust-ncg',
               jac=rosen_der, hess=rosen_hess,
               options={'gtol': 1e-8, 'disp': True})

print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 20
         Function evaluations: 21
         Gradient evaluations: 20
         Hessian evaluations: 19
[1. 1. 1. 1. 1.]


**Constrained minimization of multivariate scalar functions (minimize)**

#####As an example let us consider the constrained minimization of the Rosenbrock function:
\begin{align}
  \text{min}_{x_0,x_1} 100(x_1-x_0^2)^2 + (1-x_0)^2\\[1em]
  \text{subject to:} \\[1em]
  x_0+2x_1 ≤ 1 \\[1em]
  x_0^2+x_1 ≤ 1 \\[1em]
  x_0^2-x_1 ≤ 1 \\[1em]
  2x_0+x_1 = 1 \\[1em]
  0 \le x_0 \le 1 \\[1em]
  -0.5 \le x_1 \le 2
    \end{align}
#####This optimization problem has the unique solution \begin{align}
[x_0,x_1] = [0.4149,0.1701]
    \end{align}
for which only the first and fourth constraints are active.

**Trust-Region Constrained Algorithm (method='trust-constr')**

In [10]:
import numpy as np
from scipy.optimize import minimize

from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import NonlinearConstraint

# define the Rosenbrock function
def rosen(x):
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

# define derivatives
def rosen_der(x):
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)
    return der

# define Hessian
def rosen_hess(x):
    x = np.asarray(x)
    H = np.diag(-400*x[:-1],1) - np.diag(400*x[:-1],-1)
    diagonal = np.zeros_like(x)
    diagonal[0] = 1200*x[0]**2-400*x[1]+2
    diagonal[-1] = 200
    diagonal[1:-1] = 202 + 1200*x[1:-1]**2 - 400*x[2:]
    H = H + np.diag(diagonal)
    return H

# define an initial guess for the optimization algorithm
x0 = np.array([0.5, 0])

# define bounds constraints
bounds = Bounds([0, -0.5], [1.0, 2.0])

# define linear constraints
linear_constraint = LinearConstraint([[1, 2], [2, 1]], [-np.inf, 1], [1, 1])

# define nonlinear constraints
def cons_f(x):
    return [x[0]**2 + x[1], x[0]**2 - x[1]]
def cons_J(x):
    return [[2*x[0], 1], [2*x[0], -1]]
def cons_H(x, v):
    return v[0]*np.array([[2, 0], [0, 0]]) + v[1]*np.array([[2, 0], [0, 0]])

nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 1, jac=cons_J, hess=cons_H)

res = minimize(rosen, x0, method='trust-constr', jac=rosen_der, hess=rosen_hess,
               constraints=[linear_constraint, nonlinear_constraint],
               options={'verbose': 2}, bounds=bounds)
print(res.x)

| niter |f evals|CG iter|  obj func   |tr radius |   opt    |  c viol  |
|-------|-------|-------|-------------|----------|----------|----------|
|   1   |   1   |   0   | +6.5000e+00 | 1.00e+00 | 1.05e+01 | 0.00e+00 |
|   2   |   2   |   1   | +9.3930e-01 | 6.13e+00 | 1.23e+00 | 1.27e-01 |
|   3   |   3   |   2   | +3.8755e-01 | 6.13e+00 | 1.38e-01 | 1.11e-16 |
|   4   |   4   |   3   | +3.4360e-01 | 6.13e+00 | 7.72e-03 | 0.00e+00 |
|   5   |   4   |   3   | +3.4360e-01 | 3.07e+01 | 1.64e-02 | 0.00e+00 |
|   6   |   5   |   4   | +3.4273e-01 | 3.07e+01 | 1.60e-04 | 0.00e+00 |
|   7   |   5   |   4   | +3.4273e-01 | 1.53e+02 | 1.89e-03 | 0.00e+00 |
|   8   |   6   |   5   | +3.4272e-01 | 1.53e+02 | 2.21e-06 | 0.00e+00 |
|   9   |   6   |   5   | +3.4272e-01 | 7.67e+02 | 3.48e-04 | 0.00e+00 |
|  10   |   7   |   6   | +3.4272e-01 | 7.67e+02 | 7.54e-08 | 0.00e+00 |
|  11   |   7   |   6   | +3.4272e-01 | 3.83e+03 | 6.92e-05 | 0.00e+00 |
|  12   |   8   |   7   | +3.4272e-01 | 3.83e+03 | 